In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error
# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.orm import Session
# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

In [7]:
engine=create_engine("sqlite:///sql.sqlite")
Base = automap_base()

In [8]:
Base.prepare(engine, reflect=True)

In [9]:
Base.classes.keys()

['Measurements']

In [ ]:
Base.classes.Stations